In [1]:
"""
================================================================================
TP5: Implémentation d'un système expert médical en Python
Système Expert de Diagnostic Médical
Université Sultan My Slimane - ENSA Khouribga
Ingénierie des connaissances 2025-2026
================================================================================
"""

# ============================================================================
# PARTIE 1 : BASE DE CONNAISSANCES STATIQUE
# ============================================================================

class BaseConnaissances:
    """Base de connaissances contenant les maladies et leurs symptômes"""
    
    def __init__(self):
        # Liste des symptômes possibles
        self.symptomes_possibles = [
            'fievre', 'toux', 'mal_gorge', 'fatigue', 'courbatures',
            'mal_tete', 'eternuements', 'nez_qui_coule', 
            'perte_gout', 'difficulte_respirer'
        ]
        
        # Noms en français pour l'affichage
        self.noms_symptomes = {
            'fievre': 'fièvre',
            'toux': 'toux',
            'mal_gorge': 'mal de gorge',
            'fatigue': 'fatigue',
            'courbatures': 'courbatures',
            'mal_tete': 'mal de tête',
            'eternuements': 'éternuements',
            'nez_qui_coule': 'nez qui coule',
            'perte_gout': 'perte de goût/odorat',
            'difficulte_respirer': 'difficulté à respirer'
        }
        
        # Définition des maladies
        self.maladies = {
            'grippe': {
                'symptomes_requis': ['fievre', 'courbatures', 'fatigue'],
                'symptomes_optionnels': ['toux', 'mal_tete'],
                'symptomes_incompatibles': []
            },
            'angine': {
                'symptomes_requis': ['mal_gorge', 'fievre'],
                'symptomes_optionnels': ['fatigue', 'mal_tete'],
                'symptomes_incompatibles': []
            },
            'covid': {
                'symptomes_requis': ['fievre', 'toux', 'fatigue'],
                'symptomes_optionnels': ['perte_gout', 'difficulte_respirer', 'courbatures'],
                'symptomes_incompatibles': []
            },
            'allergie': {
                'symptomes_requis': ['eternuements', 'nez_qui_coule'],
                'symptomes_optionnels': [],
                'symptomes_incompatibles': ['fievre']
            },
            'rhume': {
                'symptomes_requis': ['toux', 'nez_qui_coule'],
                'symptomes_optionnels': ['eternuements', 'mal_gorge', 'fatigue'],
                'symptomes_incompatibles': []
            }
        }
    
    def obtenir_nom_francais(self, symptome):
        """Retourne le nom en français d'un symptôme"""
        return self.noms_symptomes.get(symptome, symptome)


# ============================================================================
# PARTIE 1.4 : TESTS AVEC PATIENTS FICTIFS (BASE STATIQUE)
# ============================================================================

class Patient:
    """Représente un patient avec ses symptômes"""
    
    def __init__(self, nom, symptomes):
        self.nom = nom
        self.symptomes = symptomes
    
    def a_symptome(self, symptome):
        """Vérifie si le patient a un symptôme donné"""
        return symptome in self.symptomes
    
    def a_tous_symptomes(self, liste_symptomes):
        """Vérifie si le patient a tous les symptômes d'une liste"""
        return all(self.a_symptome(s) for s in liste_symptomes)
    
    def a_symptome_incompatible(self, liste_incompatibles):
        """Vérifie si le patient a un symptôme incompatible"""
        return any(self.a_symptome(s) for s in liste_incompatibles)


def diagnostic_patient(patient, base_connaissances):
    """Effectue un diagnostic pour un patient donné"""
    maladies_detectees = []
    
    for maladie, infos in base_connaissances.maladies.items():
        # Vérifier si le patient a tous les symptômes requis
        if patient.a_tous_symptomes(infos['symptomes_requis']):
            # Vérifier qu'il n'a pas de symptômes incompatibles
            if not patient.a_symptome_incompatible(infos['symptomes_incompatibles']):
                maladies_detectees.append(maladie)
    
    return maladies_detectees


def tester_patients_fictifs():
    """Teste le système avec des patients fictifs"""
    print("\n" + "="*70)
    print("PARTIE 1 : TESTS AVEC PATIENTS FICTIFS")
    print("="*70 + "\n")
    
    base = BaseConnaissances()
    
    # Créer des patients de test
    patients = [
        Patient("P1", ['fievre', 'toux', 'fatigue']),
        Patient("P2", ['mal_gorge', 'fievre']),
        Patient("P3", ['eternuements', 'nez_qui_coule']),
        Patient("P4", ['fievre', 'courbatures', 'fatigue', 'mal_tete'])
    ]
    
    # Tester chaque patient
    for patient in patients:
        print(f"Patient {patient.nom}:")
        print(f"  Symptômes: {[base.obtenir_nom_francais(s) for s in patient.symptomes]}")
        
        diagnostics = diagnostic_patient(patient, base)
        
        if diagnostics:
            print(f"  Diagnostic(s) possible(s): {', '.join(diagnostics)}")
        else:
            print(f"  Aucune maladie détectée")
        print()


# ============================================================================
# PARTIE 2 : INTERACTION AVEC L'UTILISATEUR
# ============================================================================

class SystemeExpert:
    """Système expert interactif pour le diagnostic médical"""
    
    def __init__(self):
        self.base = BaseConnaissances()
        self.reponses = {}  # Mémorisation des réponses
    
    def reinitialiser(self):
        """Réinitialise les réponses mémorisées"""
        self.reponses = {}
    
    def poser_question(self, symptome):
        """Pose une question à l'utilisateur sur un symptôme"""
        nom_fr = self.base.obtenir_nom_francais(symptome)
        
        while True:
            reponse = input(f"Avez-vous {nom_fr} ? (o/n) : ").strip().lower()
            
            if reponse in ['o', 'oui', 'y', 'yes']:
                self.reponses[symptome] = True
                return True
            elif reponse in ['n', 'non']:
                self.reponses[symptome] = False
                return False
            else:
                print("⚠️  Réponse invalide. Veuillez répondre par 'o' (oui) ou 'n' (non).")
    
    def a_symptome(self, symptome):
        """Vérifie si l'utilisateur a un symptôme (avec mémorisation)"""
        # Si déjà demandé, retourner la réponse mémorisée
        if symptome in self.reponses:
            return self.reponses[symptome]
        
        # Sinon, poser la question
        return self.poser_question(symptome)
    
    def verifier_symptomes_requis(self, liste_symptomes):
        """Vérifie si tous les symptômes requis sont présents"""
        return all(self.a_symptome(s) for s in liste_symptomes)
    
    def verifier_incompatibles(self, liste_incompatibles):
        """Vérifie qu'aucun symptôme incompatible n'est présent"""
        return not any(self.a_symptome(s) for s in liste_incompatibles)
    
    def detecter_maladie(self, maladie, infos):
        """Détecte si une maladie correspond aux symptômes"""
        # Vérifier les symptômes requis
        if not self.verifier_symptomes_requis(infos['symptomes_requis']):
            return False
        
        # Vérifier les symptômes incompatibles
        if not self.verifier_incompatibles(infos['symptomes_incompatibles']):
            return False
        
        return True
    
    def trouver_maladies(self):
        """Trouve toutes les maladies compatibles avec les symptômes"""
        maladies_detectees = []
        
        for maladie, infos in self.base.maladies.items():
            if self.detecter_maladie(maladie, infos):
                maladies_detectees.append(maladie)
        
        return maladies_detectees


# ============================================================================
# PARTIE 3 : EXPLICATION (POURQUOI ?)
# ============================================================================

class Explicateur:
    """Génère des explications pour les diagnostics"""
    
    def __init__(self, systeme_expert):
        self.systeme = systeme_expert
        self.base = systeme_expert.base
    
    def symptomes_confirmes(self, maladie):
        """Récupère les symptômes confirmés pour une maladie"""
        infos = self.base.maladies[maladie]
        
        # Symptômes requis confirmés
        symptomes_req = [s for s in infos['symptomes_requis'] 
                        if self.systeme.reponses.get(s, False)]
        
        # Symptômes optionnels confirmés
        symptomes_opt = [s for s in infos['symptomes_optionnels'] 
                        if self.systeme.reponses.get(s, False)]
        
        return symptomes_req + symptomes_opt
    
    def expliquer(self, maladie):
        """Génère une explication pour une maladie"""
        symptomes = self.symptomes_confirmes(maladie)
        noms_fr = [self.base.obtenir_nom_francais(s) for s in symptomes]
        
        return f"Symptômes détectés : {', '.join(noms_fr)}"


# ============================================================================
# AFFICHAGE DES RÉSULTATS
# ============================================================================

def afficher_entete():
    """Affiche l'en-tête du système expert"""
    print("\n" + "="*70)
    print("     SYSTÈME EXPERT MÉDICAL - DIAGNOSTIC DE MALADIES")
    print("     ENSA Khouribga - Ingénierie des Connaissances")
    print("="*70)
    print("\nCe système vous aidera à identifier une maladie possible.")
    print("Répondez aux questions par 'o' (oui) ou 'n' (non).")
    print()


def afficher_resultats(maladies, explicateur):
    """Affiche les résultats du diagnostic"""
    print("\n" + "="*70)
    
    if not maladies:
        print("AUCUNE MALADIE DÉTECTÉE")
        print("="*70)
        print("\nAucune maladie correspondant à vos symptômes n'a été identifiée.")
        print("Consultez un médecin pour un diagnostic professionnel.")
    
    elif len(maladies) == 1:
        print("DIAGNOSTIC")
        print("="*70)
        maladie = maladies[0]
        print(f"\nVous pourriez avoir : {maladie.upper()}")
        print(f"\nPourquoi {maladie} ?")
        print(f"  {explicateur.expliquer(maladie)}")
        print("\n⚠️  Ce diagnostic est indicatif. Consultez un médecin pour confirmation.")
    
    else:
        print("DIAGNOSTICS POSSIBLES")
        print("="*70)
        print("\nPlusieurs maladies correspondent à vos symptômes :\n")
        
        for i, maladie in enumerate(maladies, 1):
            print(f"{i}. {maladie.upper()}")
            print(f"   {explicateur.expliquer(maladie)}")
            print()
        
        print("⚠️  Ces diagnostics sont indicatifs. Consultez un médecin pour confirmation.")
    
    print("="*70)


# ============================================================================
# FONCTION PRINCIPALE
# ============================================================================

def expert():
    """Point d'entrée principal du système expert"""
    afficher_entete()
    
    # Créer le système expert
    systeme = SystemeExpert()
    systeme.reinitialiser()
    
    # Trouver les maladies
    maladies = systeme.trouver_maladies()
    
    # Créer l'explicateur
    explicateur = Explicateur(systeme)
    
    # Afficher les résultats
    afficher_resultats(maladies, explicateur)
    
    return systeme, maladies


def nouveau_diagnostic():
    """Permet de faire plusieurs diagnostics"""
    while True:
        systeme, maladies = expert()
        
        print("\n" + "-"*70)
        reponse = input("Voulez-vous faire un nouveau diagnostic ? (o/n) : ").strip().lower()
        
        if reponse not in ['o', 'oui', 'y', 'yes']:
            print("\nMerci d'avoir utilisé le système expert. Au revoir ! 👋")
            break


# ============================================================================
# VERSION AVEC VISUALISATION (BONUS)
# ============================================================================

def afficher_resume_symptomes(systeme):
    """Affiche un résumé visuel des symptômes"""
    print("\n" + "="*70)
    print("RÉSUMÉ DE VOS SYMPTÔMES")
    print("="*70)
    
    symptomes_oui = [s for s, rep in systeme.reponses.items() if rep]
    symptomes_non = [s for s, rep in systeme.reponses.items() if not rep]
    
    if symptomes_oui:
        print("\n✓ Symptômes présents:")
        for s in symptomes_oui:
            print(f"  • {systeme.base.obtenir_nom_francais(s)}")
    
    if symptomes_non:
        print("\n✗ Symptômes absents:")
        for s in symptomes_non:
            print(f"  • {systeme.base.obtenir_nom_francais(s)}")
    
    print("="*70)


def expert_detaille():
    """Version détaillée du système expert avec résumé"""
    afficher_entete()
    
    systeme = SystemeExpert()
    systeme.reinitialiser()
    
    maladies = systeme.trouver_maladies()
    
    # Afficher le résumé des symptômes
    afficher_resume_symptomes(systeme)
    
    # Afficher les résultats
    explicateur = Explicateur(systeme)
    afficher_resultats(maladies, explicateur)
    
    return systeme, maladies


# ============================================================================
# MENU PRINCIPAL
# ============================================================================

def menu_principal():
    """Menu principal du système expert"""
    while True:
        print("\n" + "╔" + "="*68 + "╗")
        print("║  SYSTÈME EXPERT MÉDICAL - MENU PRINCIPAL" + " "*26 + "║")
        print("╚" + "="*68 + "╝")
        print("\n1. Tester avec des patients fictifs (Partie 1)")
        print("2. Diagnostic interactif simple")
        print("3. Diagnostic interactif détaillé")
        print("4. Quitter")
        
        choix = input("\nVotre choix (1-4) : ").strip()
        
        if choix == '1':
            tester_patients_fictifs()
        elif choix == '2':
            nouveau_diagnostic()
        elif choix == '3':
            while True:
                expert_detaille()
                reponse = input("\nNouveau diagnostic détaillé ? (o/n) : ").strip().lower()
                if reponse not in ['o', 'oui', 'y', 'yes']:
                    break
        elif choix == '4':
            print("\nMerci d'avoir utilisé le système expert. Au revoir ! 👋\n")
            break
        else:
            print("\n⚠️  Choix invalide. Veuillez choisir entre 1 et 4.")


# ============================================================================
# POINT D'ENTRÉE
# ============================================================================

if __name__ == "__main__":
    print("\n" + "╔" + "="*68 + "╗")
    print("║  TP5 - SYSTÈME EXPERT MÉDICAL EN PYTHON" + " "*28 + "║")
    print("║  Université Sultan My Slimane - ENSA Khouribga" + " "*21 + "║")
    print("║  Ingénierie des Connaissances 2025-2026" + " "*28 + "║")
    print("╚" + "="*68 + "╝")
    
    menu_principal()


╔====================================================================╗
║  TP5 - SYSTÈME EXPERT MÉDICAL EN PYTHON                            ║
║  Université Sultan My Slimane - ENSA Khouribga                     ║
║  Ingénierie des Connaissances 2025-2026                            ║
╚====================================================================╝

╔====================================================================╗
║  SYSTÈME EXPERT MÉDICAL - MENU PRINCIPAL                          ║
╚====================================================================╝

1. Tester avec des patients fictifs (Partie 1)
2. Diagnostic interactif simple
3. Diagnostic interactif détaillé
4. Quitter

Votre choix (1-4) : 1

PARTIE 1 : TESTS AVEC PATIENTS FICTIFS

Patient P1:
  Symptômes: ['fièvre', 'toux', 'fatigue']
  Diagnostic(s) possible(s): covid

Patient P2:
  Symptômes: ['mal de gorge', 'fièvre']
  Diagnostic(s) possible(s): angine

Patient P3:
  Symptômes: ['éternuements', 'nez qui coule']
  D